In [60]:
#Apply the Radon-Nikodym principles to analyze anomalies in ITC's daily closing prices and develop a buy and sell strategy

In [61]:
# This output includes all required data to evaluate and analyze the trading strategy.
#You can fine-tune the buy and sell thresholds to optimize the strategy to reach a Sharpe ratio of 1.85.

In [62]:
import yfinance as yf
import pandas as pd
import numpy as np

# Step 1: Pull ITC's Daily Closing Prices
stock = yf.download('BCG.NS', start='2023-05-17', end='2024-5-17')
closing_prices = stock['Close']

# Step 2: Calculate the Radon-Nikodym Derivative
returns = closing_prices.pct_change().dropna()
rolling_mean = returns.rolling(window=8).mean()
rolling_std = returns.rolling(window=15).std()
radon_nikodym_derivative = (returns - rolling_mean) / rolling_std

# Step 3: Develop a Trading Strategy
buy_threshold = -1
sell_threshold = 1
buy_signals = radon_nikodym_derivative < buy_threshold
sell_signals = radon_nikodym_derivative > sell_threshold
signals = pd.DataFrame(index=returns.index)
signals['Returns'] = returns
signals['Buy'] = buy_signals
signals['Sell'] = sell_signals

# Step 4: Optimize the Strategy for a Sharpe Ratio of 1.85
initial_capital = 100000
capital = initial_capital
position = 0
portfolio_value = initial_capital
drawdown = 0
max_drawdown = 0

trades = []
positions = []
capital_over_time = []

for i in range(len(signals)):
    if signals['Buy'][i] and capital > 0:
        buy_price = closing_prices[i]
        position = capital / buy_price
        trade = {
            'Buy Date': signals.index[i],
            'Buy Price': buy_price,
            'Sell Date': None,
            'Sell Price': None,
            'Change (%)': None,
            'Portfolio Value': None,
            'Max Drawdown': None
        }
        trades.append(trade)
        capital = 0
    elif signals['Sell'][i] and position > 0:
        sell_price = closing_prices[i]
        capital = position * sell_price
        trade = trades[-1]
        trade['Sell Date'] = signals.index[i]
        trade['Sell Price'] = sell_price
        trade['Change (%)'] = ((sell_price - trade['Buy Price']) / trade['Buy Price']) * 100
        trade['Portfolio Value'] = capital
        trades[-1] = trade
        position = 0
    
    portfolio_value = capital + position * closing_prices[i]
    drawdown = max(drawdown, (portfolio_value - initial_capital) / initial_capital * 100)
    max_drawdown = max(max_drawdown, drawdown)
    capital_over_time.append(portfolio_value)

# Calculate portfolio returns
portfolio_returns = pd.Series(capital_over_time).pct_change().dropna()
risk_free_rate = 0.03
sharpe_ratio = (portfolio_returns.mean() - risk_free_rate / 252) / portfolio_returns.std() * (252 ** 0.5)

# Print summary table
summary_table = pd.DataFrame(trades)
summary_table['Max Drawdown'] = max_drawdown
print(summary_table)
print(f'Sharpe Ratio: {round(sharpe_ratio,2)}')

[*********************100%%**********************]  1 of 1 completed
     Buy Date  Buy Price  Sell Date  Sell Price  Change (%)  Portfolio Value  \
0  2023-06-22  34.750000 2023-06-28   28.350000  -18.417265     81582.734911   
1  2023-07-10  30.549999 2023-07-14   25.000000  -18.166937     66761.650530   
2  2023-07-24  26.450001 2023-07-31   23.900000   -9.640836     60325.269421   
3  2023-08-07  25.250000 2023-08-16   24.049999   -4.752478     57458.324101   
4  2023-08-18  26.450001 2023-09-08   13.200000  -50.094520     28674.852374   
5  2023-09-22  20.200001 2023-10-12   16.950001  -16.089108     24061.324320   
6  2023-10-23  16.600000 2023-10-26   15.250000   -8.132532     22104.529364   
7  2023-11-07  17.650000 2023-11-30   16.200001   -8.215291     20288.577921   
8  2023-12-11  21.600000 2023-12-21   18.600000  -13.888889     17470.719926   
9  2024-01-15  21.000000 2024-01-24   17.700001  -15.714282     14725.321715   
10 2024-02-28  18.000000 2024-03-01   17.200001   -

C:\Users\DELL\AppData\Local\Temp\ipykernel_2548\1239114830.py:38: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\DELL\AppData\Local\Temp\ipykernel_2548\1239114830.py:52: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\DELL\AppData\Local\Temp\ipykernel_2548\1239114830.py:63: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\DELL\AppData\Local\Temp\ipykernel_2548\1239114830.py:39: FutureWarning:

Series.__getitem__

In [63]:
import yfinance as yf
import pandas as pd
import numpy as np
import vectorbt as vbt

# Step 1: Fetch stock Daily Closing Prices

closing_prices = stock['Close']

# Step 2: Generate Buy and Sell Signals
# Calculate daily returns
returns = closing_prices.pct_change().dropna()

# Use a rolling window to compute the mean and standard deviation of returns
rolling_mean = rolling_mean
rolling_std = rolling_std

# Radon-Nikodym derivative (standardized returns)
radon_nikodym_derivative = (returns - rolling_mean) / rolling_std

# Define thresholds for buy and sell signals
buy_threshold = buy_threshold
sell_threshold = sell_threshold

# Generate buy and sell signals
buy_signals = radon_nikodym_derivative < buy_threshold
sell_signals = radon_nikodym_derivative > sell_threshold

# Align lengths by trimming closing_prices to match the signal series
aligned_closing_prices = closing_prices.loc[radon_nikodym_derivative.index]

# Ensure buy_signals and sell_signals are boolean arrays
entries = buy_signals.astype(bool)
exits = sell_signals.astype(bool)

# Step 3: Backtest the Strategy using vectorbt
# Perform the backtest
pf = vbt.Portfolio.from_signals(aligned_closing_prices, entries, exits, init_cash=100000, fees=0.001)

# Output the portfolio stats
stats = pf.stats()
# Step 4: Display the Results
# Display performance metrics
print(stats)

C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2023-05-18 00:00:00
End                           2024-05-16 00:00:00
Period                                        245
Start Value                              100000.0
End Value                            26599.772478
Total Return [%]                       -73.400228
Benchmark Return [%]                   -25.080389
Max Gross Exposure [%]                      100.0
Total Fees Paid                       1571.308188
Max Drawdown [%]                        73.400228
Max Drawdown Duration                       220.0
Total Trades                                   16
Total Closed Trades                            15
Total Open Trades                               1
Open Trade PnL                       -1397.915998
Win Rate [%]                                 20.0
Best Trade [%]                           3.068708
Worst Trade [%]                        -45.626966
Avg Winning Trade [%]                    1.785447
Avg Losing Trade [%]                    -9.518612


In [64]:
# Plot the portfolio value and orders
fig = pf.plot(subplots=['value', 'orders'])
fig.show()